In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import networkx as nx
import random
import pickle
import itertools
import math
import urllib.request as urllib
import io
import zipfile

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.linalg import diag
from tensorflow.keras import callbacks
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score
from networkx.generators.community import LFR_benchmark_graph
from itertools import count
from scipy.spatial import distance_matrix

from helpers import *
from Autoencoder import *

In [ ]:
cs = pickle.load(open('citeseer.p', 'rb'))

In [ ]:
#3311-512-256-128-64

#modularity
X_cs_1 = tf.convert_to_tensor(nx.modularity_matrix(cs).astype('float32'))

In [ ]:
#train new network
ae_cs_1 = Autoencoder(3311, 512, k_reg=tf.keras.regularizers.L2(), sparse=True)
history_cs_1 = train(ae_cs_1, 5000, 3312, X_cs_1)

X_cs_2 = ae_cs_1.encoder(X_cs_1)

ae_cs_2 = Autoencoder(512, 256, k_reg=tf.keras.regularizers.L2(), sparse=True)
history_cs_2 = train(ae_cs_2, 5000, 512, X_cs_2)

X_cs_3 = ae_cs_2.encoder(X_cs_2)

ae_cs_3 = Autoencoder(256, 128, k_reg=tf.keras.regularizers.L2(), sparse=True)
history_cs_3 = train(ae_cs_3, 5000, 256, X_cs_3)

X_cs_4 = ae_cs_3.encoder(X_cs_3)

ae_cs_4 = Autoencoder(128, 64, k_reg=tf.keras.regularizers.L2(), sparse=True)
history_cs_4 = train(ae_cs_4, 5000, 128, X_cs_4)

In [ ]:
#train existing network
history_cs_1 = train(ae_cs_1, 10000, 3312, X_cs_1)

X_cs_2 = ae_cs_1.encoder(X_cs_1)

history_cs_2 = train(ae_cs_2, 10000, 512, X_cs_2)

X_cs_3 = ae_cs_2.encoder(X_cs_2)

history_cs_3 = train(ae_cs_3, 10000, 256, X_cs_3)

X_cs_4 = ae_cs_3.encoder(X_cs_3)

history_cs_4 = train(ae_cs_4, 10000, 128, X_cs_4)

In [ ]:
H_cs = ae_cs_1.encoder(X_cs_1)
labels = cora_labels(cs)
kmeans_cs = KMeans(n_clusters=6, random_state=0).fit(H_cs)

In [ ]:
normalized_mutual_info_score(labels, kmeans_cs.labels_)